# Zero-Shot Object Detection with Aya Vision

C4AI Aya Vision 8B is an open weights research release of an 8-billion parameter model with advanced capabilities optimized for a variety of vision-language use cases, including OCR, captioning, visual reasoning, summarization, question answering, code, and more. It is a multilingual model trained to excel in 23 languages in vision and language.

## Environment setup

### Configure your API keys

- Open your [`HuggingFace Settings`](https://huggingface.co/settings) page. Click `Access Tokens` then `New Token` to generate new token.


In [1]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Check GPU availability

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `T4 GPU`, and then click `Save`.

In [2]:
!nvidia-smi

Fri Mar  7 10:40:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Install dependencies

Installs all required python libraries, including [`maestro`](https://github.com/roboflow/maestro) for Qwen2.5-VL and [`supervision`](https://github.com/roboflow/supervision) for visualization.

In [3]:
! pip install 'git+https://github.com/huggingface/transformers.git@v4.49.0-AyaVision'


  Cloning https://github.com/huggingface/transformers.git (to revision v4.49.0-AyaVision) to /tmp/pip-req-build-g5_red2c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-g5_red2c
  Running command git checkout -q 512e07332568b69b219f260bf0864d8ae6156750
  Resolved https://github.com/huggingface/transformers.git to commit 512e07332568b69b219f260bf0864d8ae6156750
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10875637 sha256=15c6ef0b5f0346ae53cab7880d20cf851d57183a0883bac406800ec9f647f441
  Stored in directory: /tmp/pip-ephem-wheel-cache-jp9rf89f/wheels/c9/2c/9e/0e345af3e1ab5d203d946fe17dc722a9d792a3314af38139a1
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Unins

## Load Aya-8B model model
# Zero Shot Object Detection
Loads the Aya-8B model model (and its processor) from Hugging Face, preparing the model for inference.

In [7]:
from transformers import AutoProcessor, AutoModelForImageTextToText
import torch

model_id = "CohereForAI/aya-vision-8b"

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForImageTextToText.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.float16
)

# Format message with the aya-vision chat template
messages = [
    {"role": "user",
     "content": [
       {"type": "image", "url": "https://media.roboflow.com/notebooks/examples/dog-2.jpeg"},
        {"type": "text", "text": "Find the dog in the image and give back the bounding box coordinates in x,y,w,h"},
    ]},
    ]

inputs = processor.apply_chat_template(
    messages, padding=True, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt"
).to(model.device)

gen_tokens = model.generate(
    **inputs,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.3,
)

print(processor.tokenizer.decode(gen_tokens[0][inputs.input_ids.shape[1]:], skip_special_tokens=True))

processor_config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/178 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1747: FutureWarning: Both `num_logits_to_keep` and `logits_to_keep` are set for `Cohere2ForCausalLM.forward`. Using `logits_to_keep=0` and ignoring deprecated `num_logits_to_keep=0`.
  return forward_call(*args, **kwargs)


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.02 GiB. GPU 0 has a total capacity of 14.74 GiB of which 610.12 MiB is free. Process 10395 has 14.14 GiB memory in use. Of the allocated memory 13.81 GiB is allocated by PyTorch, and 208.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)